<a href="https://colab.research.google.com/github/osmarbraz/exemplos_BERT/blob/main/ExemplosWordEmbeddingContextualBERT_pt_br_frase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de Word Embeddings(pt-br) Contextual usando BERT Transformers by HuggingFace

## **A execução pode ser feita através do menu Ambiente de Execução opção Executar tudo.**

Exemplos de uso de **Word Embeddings Contextuais** para **desambiguação** de documentos originais e permutados. No final do notebook estão os exemplos com os documentos:

*   documento original e permutado

**Link biblioteca Huggingface:**
https://github.com/huggingface/transformers


**Artigo original BERT Jacob Devlin:**
https://arxiv.org/pdf/1506.06724.pdf

## 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

###Tratamento de logs

Método para tratamento dos logs.

In [1]:
#biblioteca de logging
import logging

#formatando a mensagem de logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [2]:
#se estiver executando no Google Colaboratory
import sys
#retorna true ou false se estiver no Google Colaboratory
IN_COLAB = 'google.colab' in sys.modules

## 1 - Instalação BERT da Hugging Face

Instala a interface pytorch para o BERT by Hugging Face. 

In [3]:
# Instala a última versão da biblioteca
#!pip install transformers

# Instala uma versão específica da biblioteca
!pip install -U transformers==4.5.1

Requirement already up-to-date: transformers==4.5.1 in /usr/local/lib/python3.7/dist-packages (4.5.1)


## 2 - Download do arquivo do PyTorch Checkpoint

Lista de modelos da comunidade:
* https://huggingface.co/models

Português(https://github.com/neuralmind-ai/portuguese-bert):  
* **'neuralmind/bert-base-portuguese-cased'**
* **'neuralmind/bert-large-portuguese-cased'**

In [4]:
# Importando as bibliotecas
import os

# Variável para setar o arquivo
URL_MODELO = None

# Comente uma das urls para carregar modelos de tamanhos diferentes(base/large)
# URL_MODELO do arquivo do modelo tensorflow
# arquivo menor(base) 1.1 Gbytes
# URL_MODELO = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip"

# arquivo grande(large) 3.5 Gbytes
URL_MODELO = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip"

# Se a variável foi setada
if URL_MODELO:

    # Diretório descompactação
    DIRETORIO_MODELO = '/content/modelo'

    # Recupera o nome do arquivo do modelo da URL_MODELO
    arquivo = URL_MODELO.split("/")[-1]

    # Nome do arquivo do vocabulário
    arquivo_vocab = "vocab.txt"

    # Caminho do arquivo na URL_MODELO
    caminho = URL_MODELO[0:len(URL_MODELO)-len(arquivo)]

    # Verifica se a pasta de descompactação existe no pasta corrente
    if not os.path.exists(DIRETORIO_MODELO):
   
        # Baixa o arquivo do modelo
        !wget $URL_MODELO
    
        # Descompacta o arquivo na pasta de descompactação
        !unzip -o $arquivo -d $DIRETORIO_MODELO

        # Baixa o arquivo do vocabulário
        # O vocabulário não está no arquivo compactado acima, mesma url mas arquivo diferente
        URL_MODELO_VOCAB = caminho + arquivo_vocab
        !wget $URL_MODELO_VOCAB
    
        # Coloca o arquivo do vocabulário no diretório de descompactação
        !mv $arquivo_vocab $DIRETORIO_MODELO
            
        # Move o arquivo para pasta de descompactação
        !mv $arquivo $DIRETORIO_MODELO
       
        print('Pasta do ' + DIRETORIO_MODELO + ' pronta!')
    else:
      print('Pasta do ' + DIRETORIO_MODELO + ' já existe!')

    # Lista a pasta corrente
    !ls -la $DIRETORIO_MODELO
else:
    print('Variável URL_MODELO não setada!')

Pasta do /content/modelo já existe!
total 2525908
drwxr-xr-x 2 root root       4096 May  6 14:32 .
drwxr-xr-x 1 root root       4096 May  6 14:32 ..
-rw-r--r-- 1 root root 1244275810 Jan 22  2020 bert-large-portuguese-cased_pytorch_checkpoint.zip
-rw-rw-r-- 1 root root        874 Jan 12  2020 config.json
-rw-rw-r-- 1 root root 1342014951 Jan 12  2020 pytorch_model.bin
-rw-r--r-- 1 root root     209528 Jan 21  2020 vocab.txt


## 3 - Carregando o Tokenizador BERT

O tokenizador utiliza WordPiece, veja em [artigo original](https://arxiv.org/pdf/1609.08144.pdf).

Carregando o tokenizador da pasta '/content/modelo/' do diretório padrão se variável `URL_MODELO` setada.

**Caso contrário carrega da comunidade**

Por default(`do_lower_case=True`) todas as letras são colocadas para minúsculas. Para ignorar a conversão para minúsculo use o parâmetro `do_lower_case=False`. Esta opção também considera as letras acentuadas(ãçéí...), que são necessárias a língua portuguesa.

O parâmetro `do_lower_case` interfere na quantidade tokens a ser gerado apartir de um texto. Quando igual a `False` reduz a quantidade de tokens gerados.

In [5]:
# Importando as bibliotecas do tokenizador
from transformers import BertTokenizer

# Se a variável URL_MODELO foi setada
if URL_MODELO:
    # Carregando o Tokenizador
    print('Carrgando o tokenizador BERT do diretório ' + DIRETORIO_MODELO + '...')

    tokenizer = BertTokenizer.from_pretrained(DIRETORIO_MODELO, 
                                              do_lower_case=False)    
else:
    # Carregando o Tokenizador da comunidade
    print('Carregando o tokenizador da comunidade...')
    
    #tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)
    tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased', 
                                              do_lower_case=False)

Carrgando o tokenizador BERT do diretório /content/modelo...


## 4 - Carregando o Modelo BERT(BertModel)

Se a variável `URL_MODELO` estiver setada carrega o modelo do diretório `content/modelo`.

Caso contrário carrega da comunidade.

Carregando o modelo da pasta '/content/modelo/' do diretório padrão.

A implementação do huggingface pytorch inclui um conjunto de interfaces projetadas para uma variedade de tarefas de PNL. Embora essas interfaces sejam todas construídas sobre um modelo treinado de BERT, cada uma possui diferentes camadas superiores e tipos de saída projetados para acomodar suas tarefas específicas de PNL.

A documentação para estas pode ser encontrada em [aqui](https://huggingface.co/transformers/v2.2.0/model_doc/bert.html).

Por default o modelo está em modo avaliação ou seja `model.eval()`.

-----------------------

Durante a avaliação do modelo, este retorna um número de diferentes objetos com base em como é configurado na chamada do método `from_pretrained`. 

Quando definimos `output_hidden_states = True` na chamada do método `from_pretrained`, retorno do modelo possui no terceiro item os estados ocultos(**hidden_states**) de todas as camadas.  Veja a documentação para mais detalhes: https://huggingface.co/transformers/model_doc/bert.html#bertmodel

Quando **`output_hidden_states = True`** model retorna:
- outputs[0] = last_hidden_state;
- outputs[1] = pooler_output; 
- outputs[2] = hidden_states.

Quando **`output_hidden_states = False`** ou não especificado model retorna:
- outputs[0] = last_hidden_state;
- outputs[1] = pooler_output.


**ATENÇÃO**: O parâmetro ´**output_hidden_states = True**´ habilita gerar as camadas ocultas do modelo. Caso contrário somente a última camada é mantida. Este parâmetro otimiza a memória mas não os resultados.


In [6]:
# Importando as bibliotecas do Modelo
from transformers import BertModel

# Se a variável URL_MODELO1 foi setada
if URL_MODELO:
    # Carregando o Tokenizador
    print('Carregando o modelo BERT do diretório ' + DIRETORIO_MODELO + '...')

    model = BertModel.from_pretrained(DIRETORIO_MODELO, 
                                      output_hidden_states = True)    
else:
    # Carregando o Tokenizador da comunidade
    print('Carregando o modelo BERT da comunidade ...')

    model = BertModel.from_pretrained('neuralmind/bert-large-portuguese-cased', 
                                       output_hidden_states = True)

Carregando o modelo BERT do diretório /content/modelo...


## 5 - Funções auxiliares

### getEmbeddingsCamadas

Funções que recuperam os embeddings das camadas:
- Primeira camada;
- Penúltima camada;
- Ùltima camada;
- Soma das 4 últimas camadas;
- Concatenação das 4 últimas camadas;
- Soma de todas as camadas.

In [7]:
def getEmbeddingPrimeiraCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
      
  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultado = output[2][0]
  # Saída: (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  
  return resultado

def getEmbeddingPenultimaCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
      
  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultado = output[2][-2]
  # Saída: (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  
  return resultado

def getEmbeddingUltimaCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
     
  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultado = output[2][-1]
  # Saída: (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  
  return resultado    

def getEmbeddingSoma4UltimasCamadas(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
      
  # Retorna todas a primeira(-1) camada
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  embeddingCamadas = output[2][-4:]
  # Saída: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  

  # Usa o método `stack` para criar uma nova dimensão no tensor 
  # com a concateção dos tensores dos embeddings.        
  #Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultadoStack = torch.stack(embeddingCamadas, dim=0)
  # Saída: <4> x <1(lote)> x <qtde_tokens> x <768 ou 1024>
  
  # Realiza a soma dos embeddings de todos os tokens para as camadas
  # Entrada: <4> x <1(lote)> x <qtde_tokens> x <768 ou 1024>
  resultado = torch.sum(resultadoStack, dim=0)
  # Saida: <1(lote)> x <qtde_tokens> x <768 ou 1024>
  
  return resultado

def getEmbeddingConcat4UltimasCamadas(output):  
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
      
  # Cria uma lista com os tensores a serem concatenados
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  # Lista com os tensores a serem concatenados
  listaConcat = []
  # Percorre os 4 últimos
  for i in [-1,-2,-3,-4]:
      # Concatena da lista
      listaConcat.append(output[2][i])
  # Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  
  # Realiza a concatenação dos embeddings de todos as camadas
  # Saída: Entrada: List das camadas(4) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultado = torch.cat(listaConcat, dim=-1)
  # Saída: Entrada: (<1(lote)> x <qtde_tokens> <3072 ou 4096>)  
    
  return resultado   

def getEmbeddingSomaTodasAsCamada(output):
  # outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
  # hidden_states é uma lista python, e cada elemento um tensor pytorch no formado <lote> x <qtde_tokens> x <768 ou 1024>.
   
  # Retorna todas as camadas descontando a primeira(0)
  # Entrada: List das camadas(13 ou 25) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  embeddingCamadas = output[2][1:]
  # Saída: List das camadas(12 ou 24) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  
  # Usa o método `stack` para criar uma nova dimensão no tensor 
  # com a concateção dos tensores dos embeddings.        
  #Entrada: List das camadas(12 ou 24) (<1(lote)> x <qtde_tokens> <768 ou 1024>)  
  resultadoStack = torch.stack(embeddingCamadas, dim=0)
  # Saída: <12 ou 24> x <1(lote)> x <qtde_tokens> x <768 ou 1024>
    
  # Realiza a soma dos embeddings de todos os tokens para as camadas
  # Entrada: <12 ou 24> x <1(lote)> x <qtde_tokens> x <768 ou 1024>
  resultado = torch.sum(resultadoStack, dim=0)
  # Saida: <1(lote)> x <qtde_tokens> x <768 ou 1024>
    
  return resultado

### Imports

In [8]:
# Import das bibliotecas
import numpy as np
import torch

import matplotlib.pyplot as plt
%matplotlib inline

### getEmbeddingsVisual

Função para gerar as coordenadas de plotagem a partir das sentenças de embeddings.

Existe uma função para os tipos de camadas utilizadas:
- Ùltima camada;
- Soma das 4 últimas camadas;
- Concatenação das 4 últimas camadas;
- Soma de todas as camadas.

In [9]:
def getEmbeddingsVisualUltimaCamada(texto, modelo, tokenizador):
    
    # Adiciona os tokens especiais
    texto_marcado = "[CLS] " + texto + " [SEP]"

    # Divide a sentença em tokens
    texto_tokenizado = tokenizador.tokenize(texto_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizador.convert_tokens_to_ids(texto_tokenizado)
    
    # Marca cada um dos tokens como pertencentes à frase "1".
    mascara_atencao = [1] * len(texto_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])
    
    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding    
    camada = getEmbeddingUltimaCamada(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, texto_tokenizado

In [10]:
def getEmbeddingsVisualSoma4UltimasCamadas(texto, modelo, tokenizador):
    
    # Adiciona os tokens especiais
    texto_marcado = "[CLS] " + texto + " [SEP]"

    # Divide a sentença em tokens
    texto_tokenizado = tokenizador.tokenize(texto_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizador.convert_tokens_to_ids(texto_tokenizado)
    
    # Marca cada um dos tokens como pertencentes à frase "1".
    mascara_atencao = [1] * len(texto_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])
    
    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding    
    camada = getEmbeddingSoma4UltimasCamadas(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, texto_tokenizado

In [11]:
def getEmbeddingsVisualConcat4UltimasCamadas(texto, modelo, tokenizador):
    
    # Adiciona os tokens especiais
    texto_marcado = "[CLS] " + texto + " [SEP]"

    # Divide a sentença em tokens
    texto_tokenizado = tokenizador.tokenize(texto_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizador.convert_tokens_to_ids(texto_tokenizado)
    
    # Marca cada um dos tokens como pertencentes à frase "1".
    mascara_atencao = [1] * len(texto_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])
    
    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding    
    camada = getEmbeddingConcat4UltimasCamadas(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, texto_tokenizado

In [12]:
def getEmbeddingsVisualSomaTodasAsCamadas(texto, modelo, tokenizador):
    
    # Adiciona os tokens especiais
    texto_marcado = "[CLS] " + texto + " [SEP]"

    # Divide a sentença em tokens
    texto_tokenizado = tokenizador.tokenize(texto_marcado)

    # Mapeia as strings dos tokens em seus índices do vocabuário    
    tokens_indexados = tokenizador.convert_tokens_to_ids(texto_tokenizado)
    
    # Marca cada um dos tokens como pertencentes à frase "1".
    mascara_atencao = [1] * len(texto_tokenizado)

    # Converte a entrada em tensores
    tokens_tensores = torch.as_tensor([tokens_indexados])
    mascara_atencao_tensores = torch.as_tensor([mascara_atencao])
    
    # Prediz os atributos dos estados ocultos para cada camada
    with torch.no_grad():        
        # Retorno de model quando ´output_hidden_states=True´ é setado:  
        #outputs[0] = last_hidden_state, outputs[1] = pooler_output, outputs[2] = hidden_states
        outputs = modelo(tokens_tensores, mascara_atencao_tensores)

    # Camada embedding    
    camada = getEmbeddingSomaTodasAsCamada(outputs)

    # Remove a dimensão 1, o lote "batches".
    token_embeddings = torch.squeeze(camada, dim=0)

    # Recupera os embeddings dos tokens como um vetor
    embeddings = token_embeddings.numpy()

    # Converte para um array
    W = np.array(embeddings)
    # Transforma em um array
    B = np.array([embeddings[0], embeddings[-1]])
    # Invertee B.T
    Bi = np.linalg.pinv(B.T)

    #Projeta a palavra no espaço
    Wp = np.matmul(Bi,W.T)

    return Wp, texto_tokenizado

## 6 - Exemplo frases de documento original e permutado

### Funções auxiliares

In [13]:
def getDocumentoTokenizado(documento, tokenizador):

    # Adiciona os tokens especiais.
    documentoMarcado = "[CLS] " + documento + " [SEP]"

    # Documento tokenizado
    documentoTokenizado = tokenizador.tokenize(documentoMarcado)

    return documentoTokenizado

In [14]:
# Localiza os índices de início e fim de uma sublista em uma lista
def encontrarIndiceSubLista(lista, sublista):
    # Recupera o tamanho da lista 
    h = len(lista)
    # Recupera o tamanho da sublista
    n = len(sublista)    
    skip = {sublista[i]: n - i - 1 for i in range(n - 1)}
    i = n - 1
    while i < h:
        for j in range(n):
            if lista[i - j] != sublista[-j - 1]:
                i += skip.get(lista[i], n)
                break
        else:
            indiceInicio = i - n + 1
            indiceFim = indiceInicio + len(sublista)-1
            return indiceInicio, indiceFim
    return -1, -1

In [15]:
def getEmbeddingFraseDocumentoEmbedding(embeddingDocumento, documento, frase, tokenizador):
  # Tokeniza o documento
  documentoTokenizado = getDocumentoTokenizado(documento, tokenizador)  
  #print(documentoTokenizado)

  # Tokeniza a frase
  fraseTokenizada = getDocumentoTokenizado(frase, tokenizador)
  
  # Remove os tokens de início e fim da frase
  fraseTokenizada.remove('[CLS]')
  fraseTokenizada.remove('[SEP]')  
  #print(fraseTokenizada)
  #print(len(fraseTokenizada))
  
  # Localiza os índices dos tokens da frase no documento
  inicio, fim = encontrarIndiceSubLista(documentoTokenizado,fraseTokenizada)
  #print(inicio,fim) 
 
  # Recupera os embeddings dos tokens da frase a partir dos embeddings do documento
  embeddingFrase = embeddingDocumento[inicio:fim+1]
  #print("embeddingFrase=", embeddingFrase.shape)
  
  # Retorna o embedding da frase no documento
  return embeddingFrase

### Documento Original

In [16]:
# Define um sentença de exemplo com diversos significados da palavra  "pilha"
documento_original = ["Bom Dia, professor.", 
             "Qual o conteúdo da prova?", 
             "Vai cair tudo na prova?",
             "Aguardo uma resposta, João."]

# Concatena as frases do documento
documento_texto_original = ' '.join(documento_original)

# Adiciona os tokens especiais
documento_marcado_original = "[CLS] " + documento_texto_original + " [SEP]"

# Divide a sentença em tokens
documento_tokenizado_original = tokenizer.tokenize(documento_marcado_original)

# Mapeia os tokens em seus índices do vocabulário
documento_tokens_indexados_original = tokenizer.convert_tokens_to_ids(documento_tokenizado_original)

# Mostra os tokens com seus índices
i = 0
for tup in zip(documento_tokenizado_original, documento_tokens_indexados_original):
    print('{:>3} {:<12} {:>6,}'.format(i, tup[0], tup[1]))
    i = i + 1

  0 [CLS]           101
  1 Bom           8,399
  2 Dia           3,616
  3 ,               117
  4 professor     2,917
  5 .               119
  6 Qual         13,082
  7 o               146
  8 conteúdo      5,015
  9 da              180
 10 prova         2,310
 11 ?               136
 12 Vai          20,805
 13 cair          9,322
 14 tudo          2,745
 15 na              229
 16 prova         2,310
 17 ?               136
 18 Agu           8,125
 19 ##ardo        2,222
 20 uma             230
 21 resposta      4,299
 22 ,               117
 23 João          1,453
 24 .               119
 25 [SEP]           102


Máscara de atenção das palavras

In [17]:
# Marca cada um dos tokens como pertencentes à frase "1".
mascara_atencao_original = [1] * len(documento_tokenizado_original)

print (mascara_atencao_original)
print (len(mascara_atencao_original))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
26


Convertendo as listas em tensores

In [18]:
# Importa a bibliteca
import torch

# Converte as entradas de listas para tensores do torch
tokens_tensores_original = torch.as_tensor([documento_tokens_indexados_original])
mascara_atencao_tensores_original = torch.as_tensor([mascara_atencao_original])

Gera os embeddings para o documento original. Guarda somente a última camada da rede em `outputs`.


In [19]:
# Prediz os atributos dos estados ocultos para cada camada
with torch.no_grad():
    # output[0] contém last_hidden_states
    outputs = model(tokens_tensores_original, mascara_atencao_tensores_original)

Recupera a saída

In [20]:
# Recupera a última e única camada da saída
last_hidden_states = outputs[0]

print ("O vetor da última camada oculta tem o formato:", last_hidden_states.size())

O vetor da última camada oculta tem o formato: torch.Size([1, 26, 1024])


Vamos nos livrar da dimensão lotes "batches", pois não precisamos dela.

In [21]:
# Remove a dimensão 1, o lote "batches".
#O método squeeze remove a primeira dimensão(0) pois possui tamanho 1
token_embeddings_original = torch.squeeze(last_hidden_states, dim=0)

print ("O vetor de tokens de embedding do documento original tem o formato:", token_embeddings_original.size())

O vetor de tokens de embedding do documento original tem o formato: torch.Size([26, 1024])


Confirmando vetores dependentes do contexto


In [22]:
for i, token_str in enumerate(documento_tokenizado_original):
  print (i, token_str)

0 [CLS]
1 Bom
2 Dia
3 ,
4 professor
5 .
6 Qual
7 o
8 conteúdo
9 da
10 prova
11 ?
12 Vai
13 cair
14 tudo
15 na
16 prova
17 ?
18 Agu
19 ##ardo
20 uma
21 resposta
22 ,
23 João
24 .
25 [SEP]


Exibe os embenddings das frases

In [23]:
# Índice das frases a serem comparadas
frase1Original = documento_original[0]
frase2Original = documento_original[1]
frase3Original = documento_original[2]
frase4Original = documento_original[3]

embeddingFrase1Original = getEmbeddingFraseDocumentoEmbedding(token_embeddings_original, documento_texto_original, frase1Original, tokenizer)
embeddingFrase2Original = getEmbeddingFraseDocumentoEmbedding(token_embeddings_original, documento_texto_original, frase2Original, tokenizer)
embeddingFrase3Original = getEmbeddingFraseDocumentoEmbedding(token_embeddings_original, documento_texto_original, frase3Original, tokenizer)
embeddingFrase4Original = getEmbeddingFraseDocumentoEmbedding(token_embeddings_original, documento_texto_original, frase4Original, tokenizer)

print('Os primeiros 4 valores de cada frase do documento original.')

print('\nFrase 1:', frase1Original,'-', str(embeddingFrase1Original[:4]))
print('Soma embedding Frase1:', frase1Original,'-', str(torch.sum(embeddingFrase1Original[:4])))

print('\nFrase 2:', frase2Original,'-', str(embeddingFrase2Original[:4]))
print('Soma embedding Frase2:', frase2Original,'-', str(torch.sum(embeddingFrase2Original[:4])))

print('\nFrase 3:', frase3Original,'-', str(embeddingFrase3Original[:4]))
print('Soma embedding Frase3:', frase3Original,'-', str(torch.sum(embeddingFrase3Original[:4])))

print('\nFrase 4:', frase4Original,'-', str(embeddingFrase4Original[:4]))
print('Soma embedding Frase4:', frase4Original,'-', str(torch.sum(embeddingFrase4Original[:4])))

Os primeiros 4 valores de cada frase do documento original.

Frase 1: Bom Dia, professor. - tensor([[-0.5098, -0.2877,  0.0873,  ...,  0.5436, -0.9302,  0.4668],
        [ 0.6078, -0.8869,  0.3736,  ..., -0.3517, -1.2140, -0.3077],
        [ 0.9075, -1.1233, -0.0093,  ...,  0.4433, -0.4633, -0.0113],
        [ 0.2499, -0.4717, -0.1217,  ...,  0.7079, -0.2300,  0.4911]])
Soma embedding Frase1: Bom Dia, professor. - tensor(-113.4554)

Frase 2: Qual o conteúdo da prova? - tensor([[-0.3987, -0.9450,  0.1785,  ...,  0.7189, -0.6772, -0.1452],
        [ 0.2067, -0.2705,  0.7145,  ...,  0.3047, -0.2718,  0.7577],
        [ 0.2355,  0.2686,  0.5669,  ...,  1.0817,  0.5614,  0.3750],
        [ 0.5264, -0.4600,  0.4810,  ..., -0.5559, -0.2941,  0.0378]])
Soma embedding Frase2: Qual o conteúdo da prova? - tensor(-115.8800)

Frase 3: Vai cair tudo na prova? - tensor([[ 0.5178,  0.0863,  0.7394,  ..., -0.5765, -0.6208, -0.2230],
        [ 0.1617,  1.1516, -0.0350,  ...,  0.1730,  0.2104, -0.0207],


Examinando os embeddings do documento original



In [24]:
# Índice das frases a serem comparadas
frase1Original = documento_original[0]
frase2Original = documento_original[1]
frase3Original = documento_original[2]
frase4Original = documento_original[3]

print("Documento Original:", documento_original)

# Localiza os índices dos tokens da frase no documento
frase1TokenizadaOriginal = tokenizer.tokenize(frase1Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,frase1TokenizadaOriginal)
embeddingFrase1Original = getEmbeddingFraseDocumentoEmbedding(token_embeddings_original, documento_texto_original, frase1Original, tokenizer)
print('\nFrase 1 Original=\'', frase1Original, '\'')
print('    Frase tokenizada:', frase1TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingFrase1Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingFrase1Original))

# Localiza os índices dos tokens da frase no documento
frase2TokenizadaOriginal = tokenizer.tokenize(frase2Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,frase2TokenizadaOriginal)
embeddingFrase2Original = getEmbeddingFraseDocumentoEmbedding(token_embeddings_original, documento_texto_original, frase2Original, tokenizer)
print('\nFrase 2 Original=\'', frase2Original, '\'')
print('    Frase tokenizada:', frase2TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingFrase2Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingFrase2Original))

# Localiza os índices dos tokens da frase no documento
frase3TokenizadaOriginal = tokenizer.tokenize(frase3Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,frase3TokenizadaOriginal)
embeddingFrase3Original = getEmbeddingFraseDocumentoEmbedding(token_embeddings_original, documento_texto_original, frase3Original, tokenizer)
print('\nFrase 3 Original=\'', frase3Original, '\'')
print('    Frase tokenizada:', frase3TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingFrase3Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingFrase3Original))

# Localiza os índices dos tokens da frase no documento
frase4TokenizadaOriginal = tokenizer.tokenize(frase4Original)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_original,frase4TokenizadaOriginal)
embeddingFrase4Original = getEmbeddingFraseDocumentoEmbedding(token_embeddings_original, documento_texto_original, frase4Original, tokenizer)
print('\nFrase 4 Original=\'', frase4Original, '\'')
print('    Frase tokenizada:', frase4TokenizadaOriginal)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingFrase4Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingFrase4Original))


Documento Original: ['Bom Dia, professor.', 'Qual o conteúdo da prova?', 'Vai cair tudo na prova?', 'Aguardo uma resposta, João.']

Frase 1 Original=' Bom Dia, professor. '
    Frase tokenizada: ['Bom', 'Dia', ',', 'professor', '.']
    => inicio em 1 e término em 5
    Formato modelo : torch.Size([5, 1024])
    Soma embeddings:  -141.45

Frase 2 Original=' Qual o conteúdo da prova? '
    Frase tokenizada: ['Qual', 'o', 'conteúdo', 'da', 'prova', '?']
    => inicio em 6 e término em 11
    Formato modelo : torch.Size([6, 1024])
    Soma embeddings:  -173.58

Frase 3 Original=' Vai cair tudo na prova? '
    Frase tokenizada: ['Vai', 'cair', 'tudo', 'na', 'prova', '?']
    => inicio em 12 e término em 17
    Formato modelo : torch.Size([6, 1024])
    Soma embeddings:  -167.58

Frase 4 Original=' Aguardo uma resposta, João. '
    Frase tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    => inicio em 18 e término em 24
    Formato modelo : torch.Size([7, 1024])
    Soma 

### Documento Permutado

In [25]:
# Define um sentença de exemplo com diversos significados da palavra  "pilha"
documento_permutado = ["Aguardo uma resposta, João.",
             "Qual o conteúdo da prova?", 
             "Bom Dia, professor.",
             "Vai cair tudo na prova?"]

# Concatena as frases do documento
documento_texto_permutado = ' '.join(documento_permutado)

# Adiciona os tokens especiais
documento_marcado_permutado = "[CLS] " + documento_texto_permutado + " [SEP]"

# Divide a sentença em tokens
documento_tokenizado_permutado = tokenizer.tokenize(documento_marcado_permutado)

# Mapeia os tokens em seus índices do vocabulário
documento_tokens_indexados_permutado = tokenizer.convert_tokens_to_ids(documento_tokenizado_permutado)

# Mostra os tokens com seus índices
i = 0
for tup in zip(documento_tokenizado_permutado, documento_tokens_indexados_permutado):
    print('{:>3} {:<12} {:>6,}'.format(i, tup[0], tup[1]))
    i = i + 1

  0 [CLS]           101
  1 Agu           8,125
  2 ##ardo        2,222
  3 uma             230
  4 resposta      4,299
  5 ,               117
  6 João          1,453
  7 .               119
  8 Qual         13,082
  9 o               146
 10 conteúdo      5,015
 11 da              180
 12 prova         2,310
 13 ?               136
 14 Bom           8,399
 15 Dia           3,616
 16 ,               117
 17 professor     2,917
 18 .               119
 19 Vai          20,805
 20 cair          9,322
 21 tudo          2,745
 22 na              229
 23 prova         2,310
 24 ?               136
 25 [SEP]           102


In [26]:
# Marca cada um dos tokens como pertencentes à frase "1".
mascara_atencao_permutado = [1] * len(documento_tokenizado_permutado)

print (mascara_atencao_permutado)
print (len(mascara_atencao_permutado))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
26


In [27]:
# Importa a bibliteca
import torch

# Converte as entradas de listas para tensores do torch
tokens_tensores_permutado = torch.as_tensor([documento_tokens_indexados_permutado])
mascara_atencao_tensores_permutado = torch.as_tensor([mascara_atencao_permutado])

In [28]:
# Prediz os atributos dos estados ocultos para cada camada
with torch.no_grad():
    # output[0] contém last_hidden_states
    outputs = model(tokens_tensores_permutado, mascara_atencao_tensores_permutado)

In [29]:
# Recupera a última e única camada da saída
last_hidden_states = outputs[0]

print ("O vetor da última camada oculta tem o formato:", last_hidden_states.size())

O vetor da última camada oculta tem o formato: torch.Size([1, 26, 1024])


In [30]:
# Remove a dimensão 1, o lote "batches".
#O método squeeze remove a primeira dimensão(0) pois possui tamanho 1
token_embeddings_permutado = torch.squeeze(last_hidden_states, dim=0)

print ("O vetor de tokens de embedding do documento permutado tem o formato:", token_embeddings_permutado.size())

O vetor de tokens de embedding do documento permutado tem o formato: torch.Size([26, 1024])


Exibe os embenddings das frases

In [31]:
# Índice das frases a serem comparadas
frase1Permutado = documento_permutado[0]
frase2Permutado = documento_permutado[1]
frase3Permutado = documento_permutado[2]
frase4Permutado = documento_permutado[3]

embeddingFrase1Permutado = getEmbeddingFraseDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, frase1Permutado, tokenizer)
embeddingFrase2Permutado = getEmbeddingFraseDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, frase2Permutado, tokenizer)
embeddingFrase3Permutado = getEmbeddingFraseDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, frase3Permutado, tokenizer)
embeddingFrase4Permutado = getEmbeddingFraseDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, frase4Permutado, tokenizer)

print('Os primeiros 4 valores de cada frase do documento permutado.')

print('\nFrase 1:', frase1Permutado,'-', str(embeddingFrase1Permutado[:4]))
print('Soma embedding Frase1:', frase1Original,'-', str(torch.sum(embeddingFrase1Original[:4])))

print('\nFrase 2:', frase2Permutado,'-', str(embeddingFrase2Permutado[:4]))
print('Soma embedding Frase2:', frase2Permutado,'-', str(torch.sum(embeddingFrase2Permutado[:4])))

print('\nFrase 3:', frase3Permutado,'-', str(embeddingFrase3Permutado[:4]))
print('Soma embedding Frase3:', frase3Permutado,'-', str(torch.sum(embeddingFrase3Original[:4])))

print('\nFrase 4:', frase4Permutado,'-', str(embeddingFrase4Permutado[:4]))
print('Soma embedding Frase4:', frase4Permutado,'-', str(torch.sum(embeddingFrase4Permutado[:4])))

Os primeiros 4 valores de cada frase do documento permutado.

Frase 1: Aguardo uma resposta, João. - tensor([[ 0.4152, -0.7838, -0.0364,  ...,  0.2686, -1.3504,  0.4707],
        [-0.6012, -1.1043, -0.2660,  ..., -0.1943, -0.8112,  0.2180],
        [ 1.2319, -1.1194,  0.9353,  ...,  0.2953, -0.1973,  0.5069],
        [ 0.8861, -0.9666,  0.1831,  ...,  0.4595, -1.1045,  0.5615]])
Soma embedding Frase1: Bom Dia, professor. - tensor(-113.4554)

Frase 2: Qual o conteúdo da prova? - tensor([[-0.2957, -1.0942, -0.0024,  ...,  0.5875, -0.7164,  0.0104],
        [ 0.2193, -0.4914,  0.7233,  ...,  0.3551, -0.3698,  0.8577],
        [ 0.1377,  0.3856,  0.6060,  ...,  1.2662,  0.4965,  0.5211],
        [ 0.5271, -0.4796,  0.4135,  ..., -0.5303, -0.3934,  0.1571]])
Soma embedding Frase2: Qual o conteúdo da prova? - tensor(-115.7949)

Frase 3: Bom Dia, professor. - tensor([[-0.2387, -0.2844,  0.3606,  ...,  0.6976, -0.9028,  0.3216],
        [ 0.8188, -0.5643,  0.6965,  ..., -0.1439, -0.7018, -0.19

In [32]:
# Índice das frases a serem comparadas
frase1Permutado = documento_permutado[0]
frase2Permutado = documento_permutado[1]
frase3Permutado = documento_permutado[2]
frase4Permutado = documento_permutado[3]

print("Documento Permutado:", documento_permutado)

# Localiza os índices dos tokens da frase no documento
frase1TokenizadaPermutado = tokenizer.tokenize(frase1Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,frase1TokenizadaPermutado)
embeddingFrase1Permutado = getEmbeddingFraseDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, frase1Permutado, tokenizer)
print('\nFrase 1 Permutada=\'', frase1Permutado, '\'')
print('    Frase tokenizada:', frase1TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingFrase1Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingFrase1Permutado))

# Localiza os índices dos tokens da frase no documento
frase2TokenizadaPermutado = tokenizer.tokenize(frase2Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,frase2TokenizadaPermutado)
embeddingFrase2Permutado = getEmbeddingFraseDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, frase2Permutado, tokenizer)
print('\nFrase 2 Permutada=\'', frase2Permutado, '\'')
print('    Frase tokenizada:', frase2TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingFrase2Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingFrase2Permutado))

# Localiza os índices dos tokens da frase no documento
frase3TokenizadaPermutado = tokenizer.tokenize(frase3Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,frase3TokenizadaPermutado)
embeddingFrase3Permutado = getEmbeddingFraseDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, frase3Permutado, tokenizer)
print('\nFrase 3 Permutada=\'', frase3Permutado, '\'')
print('    Frase tokenizada:', frase3TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingFrase3Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingFrase3Permutado))

# Localiza os índices dos tokens da frase no documento
frase4TokenizadaPermutado = tokenizer.tokenize(frase4Permutado)
inicio, fim = encontrarIndiceSubLista(documento_tokenizado_permutado,frase4TokenizadaPermutado)
embeddingFrase4Permutado = getEmbeddingFraseDocumentoEmbedding(token_embeddings_permutado, documento_texto_permutado, frase4Permutado, tokenizer)
print('\nFrase 4 Permutada=\'', frase4Permutado, '\'')
print('    Frase tokenizada:', frase4TokenizadaPermutado)
print('    => inicio em', inicio , 'e término em', fim)
print('    Formato modelo :', embeddingFrase4Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingFrase4Permutado))


Documento Permutado: ['Aguardo uma resposta, João.', 'Qual o conteúdo da prova?', 'Bom Dia, professor.', 'Vai cair tudo na prova?']

Frase 1 Permutada=' Aguardo uma resposta, João. '
    Frase tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    => inicio em 1 e término em 7
    Formato modelo : torch.Size([7, 1024])
    Soma embeddings:  -199.90

Frase 2 Permutada=' Qual o conteúdo da prova? '
    Frase tokenizada: ['Qual', 'o', 'conteúdo', 'da', 'prova', '?']
    => inicio em 8 e término em 13
    Formato modelo : torch.Size([6, 1024])
    Soma embeddings:  -173.06

Frase 3 Permutada=' Bom Dia, professor. '
    Frase tokenizada: ['Bom', 'Dia', ',', 'professor', '.']
    => inicio em 14 e término em 18
    Formato modelo : torch.Size([5, 1024])
    Soma embeddings:  -141.81

Frase 4 Permutada=' Vai cair tudo na prova? '
    Frase tokenizada: ['Vai', 'cair', 'tudo', 'na', 'prova', '?']
    => inicio em 19 e término em 24
    Formato modelo : torch.Size([6, 1024])
    

### Comparando as frases

A mesma frase apresenta embeddings com valores diferentes, pois se encontram em locais diferentes do documento. A soma de todos os embeddings demonstra isto.

In [33]:
print('\nFrase 4 Original=\'', frase4Original, '\'')
print('    Frase tokenizada:', frase4TokenizadaOriginal)
print('    Formato modelo :', embeddingFrase4Original.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingFrase4Original))
print('    Os 4 primeiros embeddings:', str(embeddingFrase4Original[:4]))

print('\nFrase 1 Permutada=\'', frase1Permutado, '\'')
print('    Frase tokenizada:', frase1TokenizadaPermutado)
print('    Formato modelo :', embeddingFrase1Permutado.shape)
print('    Soma embeddings:  %.2f' % torch.sum(embeddingFrase1Permutado))
print('    Os 4 primeiros embeddings:', str(embeddingFrase1Permutado[:4]))


Frase 4 Original=' Aguardo uma resposta, João. '
    Frase tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    Formato modelo : torch.Size([7, 1024])
    Soma embeddings:  -200.72
    Os 4 primeiros embeddings: tensor([[ 0.4339, -0.7168,  0.1173,  ...,  0.0768, -1.2036,  0.3863],
        [-0.6806, -1.0194, -0.0765,  ..., -0.1768, -0.6326,  0.2895],
        [ 1.1384, -0.8541,  0.8992,  ...,  0.2798, -0.0381,  0.5061],
        [ 0.9206, -0.9862,  0.1347,  ...,  0.4182, -1.0164,  0.4713]])

Frase 1 Permutada=' Aguardo uma resposta, João. '
    Frase tokenizada: ['Agu', '##ardo', 'uma', 'resposta', ',', 'João', '.']
    Formato modelo : torch.Size([7, 1024])
    Soma embeddings:  -199.90
    Os 4 primeiros embeddings: tensor([[ 0.4152, -0.7838, -0.0364,  ...,  0.2686, -1.3504,  0.4707],
        [-0.6012, -1.1043, -0.2660,  ..., -0.1943, -0.8112,  0.2180],
        [ 1.2319, -1.1194,  0.9353,  ...,  0.2953, -0.1973,  0.5069],
        [ 0.8861, -0.9666,  0.1831,  ...,  0.4